# Getting started with Pipeline's

Here is a little illustrative example how [scikit-learn's](https://scikit-learn.org/stable/) pipelines can be used for feature engineering and data transformation in one go. The aim of the Notebook is to show some examples how pipelines can be used, but it should *not* be considered as a tutorial on Machine Learning. 

The one advantage of using pipelines is that for all datasets test, train and validation, the exact same transformations can be done easily, in rather compact manner and without a large amount of overhead code. Also error handling and dealing with code failure can be done on a single place rather then distributed over several classes, functions and interfaces. 

In this example we gonna apply the following, common steps via via pipelines:
- Correcting Missing data via `SimpleImputer`
- Scaling the Data using `StandardScaler`
- Feature engineering by implementing two custom transformers


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from typing import List, Any

In [2]:
URL_TO_DATA = (
    "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
)
TEST_SIZE = 0.2
VALID_SIZE = 0.25
RANDOM_STATE = 42
NUMERIC_TRANSFORMER_REPLACEMENT = "median"

As an example we are using the classic Titanic survival dataset. For a detailed description how the data looks and what each colum represents please have a look on [Kaggle](https://www.kaggle.com/competitions/titanic/data). As common feature we add the Family Size and correct the Titles.

In [3]:
# in case of CERTIFICATE_VERIFY_FAILED run Install Certificates.command
# see also https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org
df = pd.read_csv(filepath_or_buffer=URL_TO_DATA, index_col=0)


df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df["Title"] = "NA"
df["Title"] = df.Name.str.extract("([A-Za-z]+)\.")

Splitting the data into test, train and validation datasets:

In [4]:
y = df["Survived"]
X = df.drop(columns=["Survived"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=VALID_SIZE, random_state=RANDOM_STATE
)  # 0.25 x 0.8 = 0.2

In [5]:
X_train.dtypes
X_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title
PassengerId,,,,,,,,,,,,
461,1,"Anderson, Mr. Harry",male,48.0,0,0,19952,26.55,E12,S,1,Mr
302,3,"McCoy, Mr. Bernard",male,NaN,2,0,367226,23.25,NaN,Q,3,Mr
386,2,"Davies, Mr. Charles Henry",male,18.0,0,0,S.O.C. 14879,73.50,NaN,S,1,Mr
321,3,"Dennis, Mr. Samuel",male,22.0,0,0,A/5 21172,7.25,NaN,S,1,Mr
346,2,"Brown, Miss. Amelia ""Mildred""",female,24.0,0,0,248733,13.00,F33,S,1,Miss


As an input for our pipeline we need to classify our columns according to type and which transformations we want to apply to them. 

In [6]:
numeric_features = ["Age", "Fare"]
categorical_features = [
    "Pclass",
    "Sex",
    "SibSp",
    "Parch",
    "Embarked",
    "Title",
    "FamilySize",
]
cutting_features = ["FamilySize"]
corrector_features = ["Title"]

BINS = [0, 1, 2, 4, np.Inf]
LABELS = ["ALONE", "SMALL", "MED", "LARGE"]


CORRECTIONS = {
    "Mlle": "Miss",
    "Mme": "Miss",
    "Ms": "Miss",
    "Dr": "Mr",
    "Major": "Mr",
    "Lady": "Mrs",
    "Countess": "Mrs",
    "Jonkheer": "Other",
    "Col": "Other",
    "Rev": "Other",
    "Capt": "Mr",
    "Sir": "Mr",
    "Don": "Mr",
}

First we take care of the numeric inputs. For filling missing values we can use [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) and provide a strategy how they should be replaced. In this example we use the median, other options provided are mean, most frequent or constant.

Scaling is another common practice for training machine learning models since the learning tends to be much harder or might be impossible if features are on completely different scales. 

In [7]:
# fmt: off
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy=NUMERIC_TRANSFORMER_REPLACEMENT)),
        ("scaler", StandardScaler()),
    ]
)
numeric_transformer
# fmt: on

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

For the categoric variables its even more simple, we just need to add one of the encoders provided in the [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) package and can directly call it as pipeline step, in our case we use a one hot encoder:

In [8]:
  
ColumnTransformer(
    transformers=[("onehot", OneHotEncoder(handle_unknown="ignore"), categorical_features)]
)


ColumnTransformer(transformers=[('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked',
                                  'Title', 'FamilySize'])])

Reading through Gunes Evitan's advanced feature engineering tutorial on Kaggle he suggested to correct the titles to a smaller set of values. The easiest way would just be using the `replace`function, but we can also do it via transformers. 

Scikit learn  provides the possibility for writing customized transformers, by simply creating a class and inherit `BaseEstimator` and `TransformerMixin`. Our class must implement `fit` and `transform` method, but since we are only using the `transform` method, `fit` will only be a placeholder. 

The transform method takes the known corrections passed while initializing the class and applies it to all columns provided to the transformer function.

Additionally we included two features which should increase usability: First the optional directly encode by providing one of scikit learns encoders provided in the prepossessing package and second the `get_feature_names_out` method, which needs do be implemented to get column names after using the custom transformer.

In [9]:
class TitleCorrector(BaseEstimator, TransformerMixin):
    """
    Use transformer to correct the Title column and do One Hot Encoding.
    """

    def __init__(
        self,
        corrections: dict,
        encoder: object = None,
    ):
        self.corrections = corrections
        self.encoder = encoder
        self.feature_names = None

    def fit(self, X, y=None) -> object:
        return self

    def transform(self, X) -> object:
        """Does the the transform step. In case an encoder is provided, encoding will be done as well."""

        X_corrected = X.apply(
            lambda x: x.replace(list(self.corrections.keys()), list(self.corrections.values()))
        )
        self.feature_names = X_corrected.columns.tolist()

        if self.encoder is not None:
            X_corrected = self.encoder.fit_transform(X_corrected)
            self.feature_names = self.encoder.get_feature_names_out()

        return X_corrected

    def get_feature_names_out(self, input_features=None) -> list:
        return self.feature_names

An other customized transformer could be for categorizing numerical variables, for example categorize the family size into alone, small medium and big. Again we can write a customized transformer for doing this. 

In [10]:
class Cutter(BaseEstimator, TransformerMixin):
    """
    Use transformer to cut numeric data. Interface to pandas:`~pandas.cut`

    """

    def __init__(self, bins: Any, labels: Any = None, encoder: object = None, **kwargs):
        self.bins = bins
        self.labels = labels
        self.kwargs = kwargs
        self.encoder = encoder
        self.feature_names = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_corrected = X.apply(
            lambda x: pd.cut(x, bins=self.bins, labels=self.labels, **self.kwargs)
        )

        self.feature_names = X_corrected.columns.tolist()

        if self.encoder is not None:
            X_corrected = self.encoder.fit_transform(X_corrected)
            self.feature_names = self.encoder.get_feature_names_out()

        return X_corrected

    def get_feature_names_out(self, input_features=None) -> list:
        return self.feature_names

As an alternative a user could simply call 
`X_test['X_corrected'] = pd.cut(X_test["FamilySize"], bins=BINS, labels=LABELS))` and or in our case, since we are using a closed dataset any how, we could apply the change before splitting. In this case the approach would be valid since the bins are fixed. But for example scaling needs to be done separate on each dataset, otherwise we would spill over information from the training data to the validation data. 

To see the beauty of of this customize transformer approach we need to imagine a production environment where every day new data flys in and retraining takes place several times. For both use cases we can apply the same code in a compact and easy way.  

As a next step we can but all our previous steps together in one pipeline: 

In [11]:
# fmt: off
one_hot_enc = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
    transformers=[
        ("discretize", Cutter(BINS, LABELS, one_hot_enc), cutting_features),
        ("correct",    TitleCorrector(CORRECTIONS, one_hot_enc), corrector_features),
        ("num", numeric_transformer, numeric_features),
        ("onehot", one_hot_enc, categorical_features),
    ]
)
preprocessor 
# fmt: on

ColumnTransformer(transformers=[('discretize',
                                 Cutter(bins=[0, 1, 2, 4, inf],
                                        encoder=OneHotEncoder(handle_unknown='ignore'),
                                        labels=['ALONE', 'SMALL', 'MED',
                                                'LARGE']),
                                 ['FamilySize']),
                                ('correct',
                                 TitleCorrector(corrections={'Capt': 'Mr',
                                                             'Col': 'Other',
                                                             'Countess': 'Mrs',
                                                             'Don': 'Mr',
                                                             'Dr': 'Mr',
                                                             'Jonkheer': 'Other',
                                                             'Lady': 'Mrs',
                                                             'Major': 'Mr',
                                                             'Mlle': 'Miss',
                                                             'Mme': 'Miss',
                                                             'Ms': 'Miss',
                                                             'Rev': 'Other',
                                                             'Sir': 'Mr'},
                                                encoder=OneHotEncoder(handle_unknown='ignore')),
                                 ['Title']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Age', 'Fare']),
                                ('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked',
                                  'Title', 'FamilySize'])])

As a next step we take a look how our data will look like after applying all pre precessing steps, therefore we execute the   

In [12]:
inspect_data = Pipeline(steps=[("preprocessor", preprocessor)])

df_to_inspect = pd.DataFrame.sparse.from_spmatrix(
    inspect_data.named_steps["preprocessor"].fit_transform(X_train)
)

df_to_inspect.columns = inspect_data["preprocessor"].get_feature_names_out()
df_to_inspect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 56 columns):
 #   Column                        Non-Null Count  Dtype             
---  ------                        --------------  -----             
 0   discretize__FamilySize_ALONE  534 non-null    Sparse[float64, 0]
 1   discretize__FamilySize_LARGE  534 non-null    Sparse[float64, 0]
 2   discretize__FamilySize_MED    534 non-null    Sparse[float64, 0]
 3   discretize__FamilySize_SMALL  534 non-null    Sparse[float64, 0]
 4   correct__Title_Master         534 non-null    Sparse[float64, 0]
 5   correct__Title_Miss           534 non-null    Sparse[float64, 0]
 6   correct__Title_Mr             534 non-null    Sparse[float64, 0]
 7   correct__Title_Mrs            534 non-null    Sparse[float64, 0]
 8   correct__Title_Other          534 non-null    Sparse[float64, 0]
 9   num__Age                      534 non-null    Sparse[float64, 0]
 10  num__Fare                     534 non-null    Spar

In [13]:
df_to_inspect.head()

,discretize__FamilySize_ALONE,discretize__FamilySize_LARGE,discretize__FamilySize_MED,discretize__FamilySize_SMALL,correct__Title_Master,correct__Title_Miss,correct__Title_Mr,correct__Title_Mrs,correct__Title_Other,num__Age,...,onehot__Title_Rev,onehot__FamilySize_1,onehot__FamilySize_2,onehot__FamilySize_3,onehot__FamilySize_4,onehot__FamilySize_5,onehot__FamilySize_6,onehot__FamilySize_7,onehot__FamilySize_8,onehot__FamilySize_11
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.477735,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.071123,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.845551,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.535780,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.380894,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Fitting the model is now rather easy, we just add a step in our pipeline and all transformers will be executed before the model gets fitted. In our case it's a simple random forest model:

In [14]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)

In [15]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_val, y_val))
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.826
model score: 0.832


#### Conclusion:

Pipelines for sure offer a nice and compact way how to execute transformations, standard scaling, and feature engineering for all datasets at once. Especially in a production environment with a complex code base this approach definitely makes sense. Especially if a lot of steps are necessary the approach provides a clear solution and provides already output for nice documentation via the HTML snippets.

Nevertheless, it might be overkill for some small use cases with closed datasets or only a few transformation steps.

#### Acknowledgement: 
- Gunes Evitan's Kaggle Notebook on [Titanic - Advanced Feature Engineering Tutorial](https://www.kaggle.com/code/gunesevitan/titanic-advanced-feature-engineering-tutorial/notebook)
- Ashwini Swain's Kaggle Notebook SWAIN [EDA To Prediction(DieTanic)](https://www.kaggle.com/ash316/eda-to-prediction-dietanic)
- Petro Morales's sklearn Tutorial on [Column Transformer with Mixed Types](https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html?highlight=standardscaler)

___
